<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_CD_2025/blob/main/IMT_CD_P4A_2025S2_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/Rogerio-mack/IMT_CD_2025/blob/main/maua_logo.png?raw=true" width=300, align="right">
<br>

In [ ]:
#@markdown imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score



# **Case**: Blood transfusion

Considere os dados abaixo sobre indivíduos potenciais doadores de sangue.

In [ ]:
df = pd.read_csv("https://github.com/Rogerio-mack/IMT_CD_2025/raw/refs/heads/main/data/blood_donate.csv")
df.head()

,Recency,Frequency,Monetary,Time,Class
0,2,50,12500,98,donated
1,0,13,3250,28,donated
2,1,16,4000,35,donated
3,2,20,5000,45,donated
4,1,24,6000,77,not donated


# Ex1. Classification

Empregue `GridSearchCV()` para avaliar SEPARADAMENTE a acuracidade dos seguintes estimadores e seus diferentes parâmetros:

<br>

> **Estimadores**

* `DecisionTreeClassifier(random_state=42)`, variando a profundidade da árvore de 5-10 e os critérios de ganho de informação 'gini','entropy','log_loss'.

* `SVC(max_iter=1000,random_state=42)`, variando todos os tipos de `kernel` disponíveis, **exceto** o `precomputed` (que requer a criação de uma função sua para `kernel`). *Ignore eventuais warnings de não convergência para o SVC*.

<br>

> **Conjuntos de Treinamento e Teste, CV**

* Empregue 25% de dados de teste, **não estratificados** com  `random_state=42` para reprodutibilidade dos resultados.

* Para o CV empregue 5 partições de CV. Os dados já estão pregue o padrão `StratifiedKFold`,`shuffle=False` (os dados já estão embaralhados).

<br>

> **Pré-Processamento dos dados**

* Empregue as melhores práticas.

<br>

> Para a reprodutibilidade dos resultados **não empregue outros parâmetros que não foram solicitados.**





In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['Class'], axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((561, 4), (561,), (187, 4), (187,))

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Q1. Quais os melhores parâmetros de cada estimador e as acuracidades obtidas no conjunto de teste?

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Q1 = []

base_estimators = {
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
    'SVC': SVC(max_iter=1000,random_state=42)
}

parm_grid = {
    'DecisionTreeClassifier': {'max_depth': range(5, 11),'criterion': ['gini','entropy','log_loss']},
    'SVC': {'kernel': ['linear', 'rbf', 'sigmoid', 'poly']}
}

for name, base_estimator in base_estimators.items():
    print(f"Training {name}...")
    clf = GridSearchCV(base_estimator, parm_grid[name], cv=5)
    clf.fit(X_train, y_train)

    print(f'best estimator: {clf.best_estimator_} with score {clf.best_score_:.3f}')
    y_pred = clf.predict(X_test)

    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test, y_pred))
    print(f'accuracy score in X_test: {clf.score(X_test, y_test):.3f}')

    Q1.append((clf.best_estimator_, np.round(clf.score(X_test, y_test),3)))


Training DecisionTreeClassifier...
best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42) with score 0.774
[[ 20  28]
 [ 23 116]]
              precision    recall  f1-score   support

     donated       0.47      0.42      0.44        48
 not donated       0.81      0.83      0.82       139

    accuracy                           0.73       187
   macro avg       0.64      0.63      0.63       187
weighted avg       0.72      0.73      0.72       187

accuracy score in X_test: 0.727
Training SVC...


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


best estimator: SVC(max_iter=1000, random_state=42) with score 0.786
[[  4  44]
 [  4 135]]
              precision    recall  f1-score   support

     donated       0.50      0.08      0.14        48
 not donated       0.75      0.97      0.85       139

    accuracy                           0.74       187
   macro avg       0.63      0.53      0.50       187
weighted avg       0.69      0.74      0.67       187

accuracy score in X_test: 0.743


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider

In [ ]:
Q1

[(DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42),
  np.float64(0.727)),
 (SVC(max_iter=1000, random_state=42), np.float64(0.743))]

## Q2. Considerando o resultado geral dos dois estimadores, qual o MELHOR estimador você selecionaria? Justifique.

In [ ]:
Q2 = str(Q1[0][0]) + ', pois apresenta acuracidade apenas ligeiramente menor que o melhor SVC, mas métricas gerais de precisão e recall melhor distribuídas.'
Q2

"DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42), pois apresenta acuracidade apenas ligeiramente menor que o melhor SVC, mas métricas gerais de precisão e recall melhor distribuídas."

# Ex2. PCA  

Empregue o melhor estimador e seus parâmetros selecionados (Q2) e conjuntos de treinamento e teste, e treine um novo modelo empregando agora apenas Componentes Principais que correspondam a no mínimo 80% da variância dos dados.



In [ ]:
from sklearn.decomposition import PCA

pca = PCA(0.8)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print(f"Número de componentes principais retidos: {pca.n_components_} com {pca.explained_variance_ratio_} da variância dos dados.")


Número de componentes principais retidos: 2 com [0.63638612 0.2739226 ] da variância dos dados.


In [ ]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42)
clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_test_pca)

print(classification_report(y_test, y_pred))
print(f'accuracy score in X_test: {clf.score(X_test_pca, y_test):.3f}')



              precision    recall  f1-score   support

     donated       0.43      0.27      0.33        48
 not donated       0.78      0.88      0.82       139

    accuracy                           0.72       187
   macro avg       0.61      0.57      0.58       187
weighted avg       0.69      0.72      0.70       187

accuracy score in X_test: 0.722


In [ ]:
Q3 = (pca.n_components_, np.round(np.sum(pca.explained_variance_ratio_),3), np.round(clf.score(X_test_pca, y_test),3) )

## Q3. Quantos componentes principais foram empregados, a variância acumulada desses componentes (treinamento) e acuracidade obtida no conjunto de teste?

In [ ]:
Q3

(np.int64(2), np.float64(0.91), np.float64(0.722))

# Ex3. Clustering

Aplique a Clusterização Hierárquica aos dados preditores empregados antes. Empregue o critério da silhueta média para obter o melhor número de clusters. Inspecione a Clusterização para as métricas `ward` e `complete`.





In [ ]:
from sklearn.preprocessing import StandardScaler

X = df.drop(['Class'], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

for link in ['complete','ward']:
  for k in range(2,5):
    hclust = AgglomerativeClustering(n_clusters=k,linkage=link)
    hclust.fit(X_scaled)
    print(f'Hclust {link}, {hclust.n_clusters_} grupos,  silhouette_score average: {silhouette_score(X_scaled, hclust.labels_):.3f}, com {sum(hclust.labels_==0)} e {sum(hclust.labels_==1)} elementos em cada grupo')
  print()


Hclust complete, 2 grupos,  silhouette_score average: 0.751, com 740 e 8 elementos em cada grupo
Hclust complete, 3 grupos,  silhouette_score average: 0.715, com 738 e 8 elementos em cada grupo
Hclust complete, 4 grupos,  silhouette_score average: 0.460, com 660 e 78 elementos em cada grupo

Hclust ward, 2 grupos,  silhouette_score average: 0.386, com 249 e 499 elementos em cada grupo
Hclust ward, 3 grupos,  silhouette_score average: 0.384, com 499 e 241 elementos em cada grupo
Hclust ward, 4 grupos,  silhouette_score average: 0.359, com 241 e 263 elementos em cada grupo



## Q4. Qual o número de clusters e método parece ser o melhor a ser empregado e por quê?

In [ ]:
Q4 =('3 grupos, maior silhueta média.','HClust com linkage ward, apresenta grupos bastante melhor distribuídos.')

## Q5. Quais cluster kmeans estão associados os indivíduos 0, 5, 6 e 8?

In [ ]:
hclust = AgglomerativeClustering(n_clusters=3,linkage='ward')
hclust.fit(X_scaled)
print(f'Hclust {link}, {3} grupos,  silhouette_score average: {silhouette_score(X_scaled, hclust.labels_):.3f}, com {sum(hclust.labels_==0)} e {sum(hclust.labels_==1)} elementos em cada grupo')

Hclust ward, 3 grupos,  silhouette_score average: 0.384, com 499 e 241 elementos em cada grupo


In [ ]:
hclust.labels_[[0, 5, 6, 8]]

array([2, 0, 0, 1])

In [ ]:
Q5 = str(hclust.labels_[[0, 5, 6, 8]])
Q5


'[2 0 0 1]'

# Ex4. Acuracidade de Clusterização



## Q6. Considere o mesmo estimador de clusterização empregado, aplicado agora para apenas 2 grupos. Qual a *acuracidade de clusterização* com relação à classe real dos dados?

In [ ]:
hclust = AgglomerativeClustering(n_clusters=2,linkage='ward')
hclust.fit(X_scaled)


AgglomerativeClustering()

In [ ]:
from sklearn.metrics import accuracy_score

print(confusion_matrix(df.Class.map({'not donated':0, 'donated':1}), hclust.labels_, labels=[0, 1]))

accuracy = accuracy_score(df.Class.map({'not donated':0, 'donated':1}), hclust.labels_)
print(f"\nAcurácia de Clusterização: {accuracy:.3f}")
print()

print(confusion_matrix(df.Class.map({'not donated':1, 'donated':0}), hclust.labels_, labels=[0, 1]))

accuracy = accuracy_score(df.Class.map({'not donated':1, 'donated':0}), hclust.labels_)
print(f"\nAcurácia de Clusterização: {accuracy:.3f}")


Q6 = f"Acurácia de Clusterização: {accuracy:.3f}"

[[177 393]
 [ 72 106]]

Acurácia de Clusterização: 0.378

[[ 72 106]
 [177 393]]

Acurácia de Clusterização: 0.622


In [ ]:
Q6

'Acurácia de Clusterização: 0.622'

In [ ]:
print('*** Gabarito ***')

for i in range(1,7):
  print()
  print(f'Q{i}. {eval(f"Q{i}")}')


*** Gabarito ***

Q1. [(DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42), np.float64(0.727)), (SVC(max_iter=1000, random_state=42), np.float64(0.743))]

Q2. DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42), pois apresenta acuracidade apenas ligeiramente menor que o melhor SVC, mas métricas gerais de precisão e recall melhor distribuídas.

Q3. (np.int64(2), np.float64(0.91), np.float64(0.722))

Q4. ('3 grupos, maior silhueta média.', 'HClust com linkage ward, apresenta grupos bastante melhor distribuídos.')

Q5. [2 0 0 1]

Q6. Acurácia de Clusterização: 0.622
